In [1]:
import sys
import datetime, time
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


def get_null_perc(spark, df, null_cols):
    """ Get null/empty percentage for columns

    Args:
        spark (Spark): SparkSession object
        df (DataFrame): dataframe to perform null/empty analysis on
        null_cols (List): list of columns that need to be considered for analysis

    Returns:
        DataFrame: dataframe with null check analysis
    """
    schema = StructType([ \
        StructField("Column",StringType(),True), \
        StructField("NullPercentage",StringType(),True)
      ])
    emptyRDD = spark.sparkContext.emptyRDD()
    resultdf = spark.createDataFrame(emptyRDD, schema=schema)
    
    for x in null_cols:
    	df_null_count = df.select(F.col(x)).filter(F.col(x).isNull() | (F.col(x) == '')).count()
    	df_null = spark.createDataFrame([[x, str(df_null_count*100.0/df.count()) + '%' ]],schema=schema)
    	resultdf = resultdf.union(df_null)

    return resultdf

def get_summary_numeric(df, numeric_cols):
    """ Get Summary for numeric columns

    Args:
        df (DataFrame): dataframe to perform analysis on
        numeric_cols (List): list of columns that need to be considered for analysis

    Returns:
        DataFrame: dataframe with summary analysis
    """

    return df.select(numeric_cols).summary()

def get_distinct_counts(spark, df, aggregate_cols):
    """ Get distinct count for columns

    Args:
        spark (Spark): SparkSession object
        df (DataFrame): dataframe to perform distinct count analysis on
        aggregate_cols (List): list of columns that need to be considered for analysis

    Returns:
        DataFrame: dataframe with distinct count analysis
    """
    schema = StructType([ \
        StructField("Column",StringType(),True), \
        StructField("DistinctCount",StringType(),True)
      ])
    
    emptyRDD = spark.sparkContext.emptyRDD()
    resultdf = spark.createDataFrame(emptyRDD, schema=schema)
    
    for x in aggregate_cols:
    	df_distinct_count = df.select(F.col(x)).distinct().count()
    	df_distinct = spark.createDataFrame([[x, str(df_distinct_count)]],schema=schema)
    	resultdf = resultdf.union(df_distinct)

    return resultdf

def get_distribution_counts(spark, df, aggregate_cols):
    """ Get Distribution Counts for columns

    Args:
        spark (Spark): SparkSession object
        df (DataFrame): dataframe to perform null/empty analysis on
        aggregate_cols (List): list of columns that need to be considered for analysis

    Returns:
        Array: Array of objects with dataframes
    """
    result = []
    for i in aggregate_cols:
    	result.append(df.groupby(F.col(i)).count().sort(F.col("count").desc()))
    ###
    
    return result

def get_mismatch_perc(spark, df, data_quality_cols_regex):
    """ Get Mismatch Percentage for columns

    Args:
        spark (Spark): SparkSession object
        df (DataFrame): dataframe to perform null/empty analysis on
        data_quality_cols_regex (Dictionary): Dictionary of columns/regex-expression for data quality analysis

    Returns:
        DataFrame: DataFrame with data quality analysis
    """
    schema = StructType([ \
        StructField("Column",StringType(),True), \
        StructField("MismatchPercentage",StringType(),True)
      ])
    
    emptyRDD = spark.sparkContext.emptyRDD()
    resultdf = spark.createDataFrame(emptyRDD, schema=schema)
    
    
    for key, value in data_quality_cols_regex.items():
    	df_regex_not_like_count = df.select(F.col(key)).filter(~F.col(key).rlike(value)).count()
    	df_regex_not_like = spark.createDataFrame([[key, str(df_regex_not_like_count*100.0/df.count()) + '%']],schema=schema)
    	resultdf = resultdf.union(df_regex_not_like)
    
    return resultdf

StatementMeta(SparkPool01, 12, 1, Finished, Available)

In [3]:
# Load tables for data quality checks 

df = spark.read.load('abfss://wwi-02@asadatalakemarziadmin.dfs.core.windows.net/customer-info/customerinfo.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(SparkPool01, 12, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, c70c775b-f7eb-4768-87de-3b3bc96411f3)

In [6]:
schema = StructType([
		 	StructField(name='Gender', dataType=StringType(), nullable=True),
		 	StructField(name='Email', dataType=StringType(), nullable=True),
            StructField(name='Phone', dataType=IntegerType(), nullable=True)
		 	])

StatementMeta(SparkPool01, 12, 6, Finished, Available)

In [ ]:
# This is a master schema of all columns name and type. below is an example for Sales.Orders Table
schema = StructType([
		 	StructField(name='OrderID', dataType=IntegerType(), nullable=False),
		 	StructField(name='CustomerID', dataType=IntegerType(), nullable=False),
		 	StructField(name='SalespersonPersonID', dataType=IntegerType(), nullable=True),
            StructField(name='PickedByPersonID', dataType=IntegerType(), nullable=True),
            StructField(name='ContactPersonID', dataType=IntegerType(), nullable=True),
            StructField(name='BackorderOrderID', dataType=IntegerType(), nullable=True),
		 	StructField(name='OrderDate', dataType=DateType(), nullable=False),
		 	StructField(name='ExpectedDeliveryDate', dataType=DateType(), nullable=True),
		 	StructField(name='CustomerPurchaseOrderNumber', dataType=StringType(), nullable=True),
            StructField(name='IsUndersupplyBackordered', dataType=StringType(), nullable=True),
            StructField(name='Comments', dataType=StringType(), nullable=True),
		 	StructField(name='DeliveryInstructions', dataType=StringType(), nullable=True),
            StructField(name='InternalComments', dataType=StringType(), nullable=True),
		 	StructField(name='PickingCompletedWhen', dataType=TimestampType(), nullable=True),
		 	StructField(name='LastEditedBy', dataType=IntegerType(), nullable=True),
		 	StructField(name='LastEditedWhen', dataType=TimestampType(), nullable=True)
		 	])

In [11]:
null_cols = ['Email']
numeric_cols = ['Phone']
aggregate_cols = ['Email']
data_quality_cols_regex = {'age': '^[0-99]{1,2}$', 'first_name': '^[a-zA-Z]*$', 'gender': '^M(ale)?$|^F(emale)?$', 'Email':'/^([a-zA-Z0-9._%-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,6})*$/'}
result_limit = 10

StatementMeta(SparkPool01, 12, 11, Finished, Available)

In [15]:
null_cols = ['OrderID','CustomerID','OrderDate']
numeric_cols = ['OrderID', 'CustomerID','SalespersonPersonID','PickedByPersonID','ContactPersonID','BackorderOrderID']
aggregate_cols = ['OrderID',]
#data_quality_cols_regex = {'age': '^[0-99]{1,2}$', 'first_name': '^[a-zA-Z]*$', 'gender': '^M(ale)?$|^F(emale)?$', 'Email':'/^([a-zA-Z0-9._%-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,6})*$/'}
data_quality_cols_regex = {'Gender': '^M(ale)?$|^F(emale)?$', 'Email':'/^([a-zA-Z0-9._%-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,6})*$/'}
result_limit = 10

StatementMeta(SparkPool01, 12, 15, Finished, Available)

In [8]:
### 1. NULL Checks
resultdf = get_null_perc(spark, df, null_cols)
print("NULL/Empty Percentage for Columns")
resultdf.show(result_limit, False)

StatementMeta(SparkPool01, 12, 8, Finished, Available)

NULL/Empty Percentage for Columns
+------+--------------+
|Column|NullPercentage|
+------+--------------+
|Email |0.0%          |
+------+--------------+



In [9]:
###2. Summary, Average, Standard Deviation, Percentiles for Numeric Columns
resultdf = get_summary_numeric(df, numeric_cols)
print("Summary for Numeric Columns")
resultdf.show(result_limit, False)

StatementMeta(SparkPool01, 12, 9, Finished, Available)

Summary for Numeric Columns
+-------+-------------------+
|summary|Phone              |
+-------+-------------------+
|count  |110                |
|mean   |null               |
|stddev |null               |
|min    |(218) 273-8495 x198|
|25%    |null               |
|50%    |null               |
|75%    |null               |
|max    |995-366-5180       |
+-------+-------------------+



In [12]:
###3. Distinct Count
print("Distinct Counts for Aggregate Columns")
resultdf = get_distinct_counts(spark, df, aggregate_cols)
resultdf.show(result_limit, False)

StatementMeta(SparkPool01, 12, 12, Finished, Available)

Distinct Counts for Aggregate Columns
+------+-------------+
|Column|DistinctCount|
+------+-------------+
|Email |110          |
+------+-------------+



In [13]:

###4. Distribution Count
print("Distribution Count for Aggregate Columns")
result = get_distribution_counts(spark, df, aggregate_cols)
for i in result:
	print("======== Distribution for - " + i.columns[0] + " ========")
	i.show(result_limit, False)


StatementMeta(SparkPool01, 12, 13, Finished, Available)

Distribution Count for Aggregate Columns
======== Distribution for - Email ========
+--------------------------------------+-----+
|Email                                 |count|
+--------------------------------------+-----+
|Noel55@adventureworks.com             |1    |
|Frank_Weissnat@adventureworks.com     |1    |
|Maria23@adventureworks.com            |1    |
|Alton_Vandervort@adventureworks.com   |1    |
|Damon.Price@adventureworks.com        |1    |
|Brian_Stark95@adventureworks.com      |1    |
|Terrell54@adventureworks.com          |1    |
|Alicia_Nicolas78@adventureworks.com   |1    |
|Charles_Breitenberg@adventureworks.com|1    |
|Alvin_Volkman36@adventureworks.com    |1    |
+--------------------------------------+-----+
only showing top 10 rows



In [16]:
###5. Data Quality
print("Data Quality Issue Percentage for Columns")
resultdf = get_mismatch_perc(spark, df, data_quality_cols_regex)
resultdf.show(result_limit, False)

StatementMeta(SparkPool01, 12, 16, Finished, Available)

Data Quality Issue Percentage for Columns
+------+------------------+
|Column|MismatchPercentage|
+------+------------------+
|Gender|0.0%              |
|Email |100.0%            |
+------+------------------+

